Notebook to see the effect of the different parameters of the Ranking Loss function mentionned in [Bi-RNN with ranking loss for spoken language understanding](http://mirlab.org/conference_papers/International_Conference/ICASSP%202016/pdfs/0006060.pdf)

> **4.2 Ranking Loss**
>
> Instead of using the sofmax activation function, we train a matrix $W_{class}$ whose columns contain vector representation of the different classes. Therefore, the score for each class *c* can be computed by using the product
>
> $$s_{\theta}(x)_c=h_x^t[W^{class}]_c \qquad (5)$$
>
>We use a ranking loss function to train the RNN. It learns to maximize the distance between the true label $y^+$ and the best competitive label $c^−$ given a data point *x*. The objective function is
>
>$$L = log(1 + exp(\gamma(m^+ - s_{\theta}(x)_{y^+})) + log(1 + exp(\gamma(m^- + s_{\theta}(x)_{c^-}))\qquad(6)$$
>
>with $s_{\theta}(x)_{y^+}$ and $s_{\theta}(x)_{c^-}$ being the scores for the classes $y^+$ and $c^−$ respectively. This function was proposed by Dos Santos et al. [[23](https://arxiv.org/pdf/1504.06580.pdf)] to train convolution neural networks for relation classification. The parameter $\gamma$ controls the penalization of the prediction errors and $m^+$ and $m^−$ are margins for the correct and incorrect classes. $\gamma$, $m^+$ and $m^−$ are hyperparameters which can be tuned on the development set. For the class O, we only calculate the second summand of equation 6

From Dos Santos [Classifying Relations by Ranking with Convolutional Neural Networks](https://arxiv.org/pdf/1504.06580.pdf)
>$$L = log(1 + exp(\gamma(m^+ - s_{\theta}(x)_{y^+})) + log(1 + exp(\gamma(m^- + s_{\theta}(x)_{c^-}))\qquad(1)$$
>
> The first term in the right side of Equation 1 decreases as the score $s_{\theta}(x)_{y^+}$ increases. The second term in the right side decreases as the score $s_{\theta}(x)_{c^-}$ decreases. Training CR-CNN by minimizing the loss function in Equation 1 **has the effect of training to give scores greater than $m^+$ for the correct class and (negative) scores smaller than $−m^−$ for incorrect classes**. In our experiments we set $\gamma$ to 2, $m^+$ to 2.5 and $m^−$ to 0.5. [emphasis added]

In [1]:
import numpy as np
from ipywidgets import interact, interactive, fixed, FloatSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Latex
plt.style.use('ggplot')

### Looking at the two parts separately.

#### Left part (true class)

In [2]:
@interact(
    gamma=FloatSlider(min=0.0, max=5.0, continuous_update=False, value=2), 
    m_plus=FloatSlider(min=0.0, max=5.0, continuous_update=False,value=2.5)
)
def g(gamma, m_plus):
    
    X = np.linspace(-10, 10, 1000)
    loss = np.log(1 + np.exp(gamma * (m_plus - X)))

    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(111)
    ax.plot(X, loss)
    ax.set_ylabel('Loss')
    ax.set_xlabel('$s_{\\theta}(x)_{y^+}$')
    ax.set_title('$log(1 + exp(\gamma(m^+ - s_{\\theta}(x)_{y^+}))$')

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='gamma', max=5.0), FloatSlid…

Playing with this graph shows that there is an inflexion point for the loss around $m^+$. When the score for the true class is higher than this point, the loss goes rapidly to zero. So the model will learn to push the the score for the true class to something higher than $m^+$

Changing $\gamma$ has the effect of a sharper inflexion point. The higher it is, the sharper it is.

#### Right part (best competing class)

In [3]:
@interact(
    gamma=FloatSlider(min=0.0, max=5.0, continuous_update=False, value=2), 
    m_neg=FloatSlider(min=0.0, max=5.0, continuous_update=False, value=0.5), 
)
def g(gamma, m_neg):
    
    X = np.linspace(-10, 10, 1000)
    loss = np.log(1 + np.exp(gamma * (m_neg + X)))
    
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(111)
    ax.plot(X, loss)
    ax.set_ylabel('Loss')
    ax.set_xlabel('$s_{\\theta}(x)_{c^-}$')
    ax.set_title('$log(1 + exp(\gamma(m^- + s_{\\theta}(x)_{c^-}))$')

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='gamma', max=5.0), FloatSlid…

The behaviour of the right part of the equation is the same as the left part but mirrored.The model will learn to push the the score for the best competitive class to something lower than $m^-$.

#### Combining both

Joining the two parts of the equation, we will get a low loss when both the true class has a high score and the best competitive class has a low score.

In [4]:
@interact(
    gamma=FloatSlider(min=0.0, max=5.0, continuous_update=False, value=2), 
    m_plus=FloatSlider(min=0.0, max=5.0, continuous_update=False,value=2.5), 
    m_neg=FloatSlider(min=0.0, max=5.0, continuous_update=False, value=0.5), 
)
def g(gamma, m_plus, m_neg):
    
    X1, X2 = np.meshgrid(np.linspace(-10, 10, 1000), np.linspace(-10, 10, 1000))
    loss = np.log(1 + np.exp(gamma * (m_plus - X1))) + np.log(1 + np.exp(gamma * (m_neg + X2)))
    
    fig = plt.figure(figsize=(15,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X1, X2, loss)
    ax.set_zlabel('Loss')
    ax.set_xlabel('$s_{\\theta}(x)_{y^+}$')
    ax.set_ylabel('$s_{\\theta}(x)_{c^-}$')

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='gamma', max=5.0), FloatSlid…

What classes to use during training? $y^+$ is the correct label (e.g. B-dest for slot filling), but $c^-$ is the *best competitive label*.

> It learns to maximize the distance between the true label $y^+$ and the best competitive label $c^−$ given a data point *x*

During training the correct class is obviously selected, but the *best competitive label* will change from one run to the other. It would have to be the highest score (excluding the true label $y^+$).

Implementing this means that you will need to check what (wrong) class has the highest score and use that in the loss function. This requires logic at training time, so Keras is out of the question. PyTorch can do that. Expecting that this will impact running time.

Last public version of PyTorch doesn't seem to have argsort, but there is a function definition for this (in a later release probably)
https://github.com/pytorch/pytorch/blob/master/torch/functional.py#L571

This should do the trick
```Python
argsort = torch.sort(scores, dim=-1, descending=True)[1]
(first, second) = argsort[:2]
best_competitive_label = second if true_label == first else first
```
The authors use SGD, so average the losses over a mini batch.

Why this loss function?

> One of the advantages of this loss function over the softmax function is efficiency. Since only two classes are computed at every training iteration, the network can be trained quite fast even with a large number of classes. Furthermore, a ranking loss function is suitable for tasks like slot filling because **it does not force the network to learn a pattern for the O class which in fact may not exist**. [emphasis added]

Why would it not learn a pattern for the O class?

> As stated in the paper *"For the class O, we only calculate the second summand of equation 6"*. The second summand is 
>
> $$log(1 + exp(\gamma(m^- + s_{\theta}(x)_{c^-}))$$
>
> This is the score for the best competitive class. By focusing on the second summand only we tell the model to learn to give low scores to every other classes while not telling it to give a high value (first summand) for the O class.

When to predict class O?

> *During testing, the model will predict class O if the score for all the other classes is lower than 0.*